In [1]:
import argparse
import os
import numpy as np
from tqdm import tqdm
import faiss
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from dataloader import Data, DataQuery
from model import Extractor, MemoryBlock
from argument_parser import add_base_args, add_eval_args
from utils import split_labels,  compute_NDCG, get_target_attr
import constants as C

In [2]:
#attr_nums = [16, 17, 19, 14, 10, 15, 2, 11, 16, 7, 9, 15]
attr_num = [37, 81, 84]

In [3]:
model = Extractor(attr_num, backbone='alexnet', dim_chunk=340)

In [4]:
model.load_state_dict(torch.load('../models/DeepFashion/extractor_best.pkl'))

<All keys matched successfully>

In [5]:
file_root = "/home/jameslee/cmu/large-scale/fashion-iq/hm_data/"
img_root_path = "/home/jameslee/cmu/large-scale/fashion-iq/hm_data/images/"

In [6]:
gallery_data = Data(file_root, img_root_path,
                        transforms.Compose([
                            transforms.Resize((C.TARGET_IMAGE_SIZE, C.TARGET_IMAGE_SIZE)),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=C.IMAGE_MEAN, std=C.IMAGE_STD)
                        ]), mode='test')

In [7]:
gallery_loader = torch.utils.data.DataLoader(gallery_data, batch_size=64, shuffle=False,
                                     sampler=torch.utils.data.SequentialSampler(gallery_data),
                                     num_workers=16,
                                     drop_last=False)

In [8]:
model.cuda()
model.eval()

Extractor(
  (backbone): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
    

In [12]:
torch.cuda.is_available()

True

In [13]:
torch.cuda.device_count()

8

In [37]:
gallery_feat = []
gallery_attr = []
with torch.no_grad():
    for i, (img, _) in enumerate(tqdm(gallery_loader)):
        img = img.cuda()
        dis_feat, attr_out = model(img)
        gallery_feat.append(F.normalize(torch.cat(dis_feat, 1)).squeeze().cpu().numpy())
        gallery_attr.append(F.normalize(torch.cat(attr_out, 1)).squeeze().cpu().numpy())


 18%|█████████▊                                             | 292/1643 [00:43<03:52,  5.80it/s]


 34%|██████████████████▌                                    | 553/1643 [01:20<02:44,  6.63it/s]


 50%|███████████████████████████▎                           | 817/1643 [01:57<02:16,  6.06it/s]


 68%|████████████████████████████████████▉                 | 1125/1643 [02:41<01:49,  4.75it/s]


 88%|███████████████████████████████████████████████▌      | 1449/1643 [03:26<00:30,  6.45it/s]


100%|██████████████████████████████████████████████████████| 1643/1643 [03:53<00:00,  7.04it/s]


In [41]:
gallery_attr[0].shape

(64, 202)

In [42]:
feats = gallery_attr[0]
for i in range(1, len(gallery_attr)):
    feats = np.vstack((feats, gallery_attr[i]))
print(feats.shape)

(105100, 202)


In [43]:
np.save("visual_features.npy", feats)

In [52]:
confidence = np.load("visual_features.npy")

In [55]:
conf_1 = confidence[:, :attr_num[0]]
conf_2 = confidence[:, attr_num[0]:attr_num[0]+attr_num[1]]
conf_3 = confidence[:, attr_num[0]+attr_num[1]:]

conf_1.shape, conf_2.shape, conf_3.shape

((105100, 37), (105100, 81), (105100, 84))

In [59]:
feat_1 = np.argmax(conf_1, axis=1)
feat_2 = np.argmax(conf_2, axis=1)
feat_3 = np.argmax(conf_3, axis=1)

In [61]:
vis_feat = np.stack((feat_1, feat_2, feat_3), axis=-1)
print(vis_feat.shape)

(105100, 3)


In [47]:
f = open("/home/jameslee/cmu/large-scale/fashion-iq/hm_data/imgs_test.txt", "r")
Lines = f.readlines()

In [64]:
Lines = [i.split(".")[0] for i in Lines]
print(Lines[0])

0685601024


In [66]:
print(len(Lines))
article_ids = np.array(Lines)

105100


In [50]:
import pandas as pd

In [68]:
df = pd.DataFrame({'article_id':article_ids, 'vis_feat_1':feat_1, 'vis_feat_2':feat_2, 'vis_feat_3':feat_3})

In [73]:
df.to_csv("visual_feat_attr.csv", index=False)